In [3]:
#=
Model: MNA_5 (10913 variables, 9 inputs)
=#
using Reachability, MAT

mna5(o::Pair{Symbol, <:Any}...) = mna5(Options(Dict{Symbol, Any}(o)))

function mna5(input_options::Options)
    # =====================
    # Problem specification
    # =====================
    file = matopen("mna5.mat")
    A = sparse(read(file, "A"))

    # initial set:
    X0 = Hyperrectangle([fill(0.000225, 10); zeros(10903)],
                        [fill(0.000025, 10); zeros(10903)])

    # input set
    B = sparse(19:27, 1:9, fill(-1., 9), size(A, 1), 9)
    U = B * Hyperrectangle([fill(0.1, 5); fill(0.2, 4)], zeros(9))

    # instantiate continuous LTI system
    S = ContinuousSystem(A, X0, U)

    # property: x1 < 0.2 && x2 < 0.15
    property = LinearConstraintProperty([
        Clause([LinearConstraint(sparsevec([1], [1.0], 10913), 0.2)]),
        Clause([LinearConstraint(sparsevec([2], [1.0], 10913), 0.15)])])

    # =======================
    # Problem default options
    # =======================
    # 2D blocks except for the last block, which is 1D
    partition = vcat([(2*i-1:2*i) for i in 1:5456], [10913:10913])

    if input_options[:mode] == "reach"
        problem_options = Options(:vars => [1],
                                  :partition => partition,
                                  :plot_vars => [0, 1],
                                  :assume_sparse => true,
                                  :lazy_expm => true)

    elseif input_options[:mode] == "check"
        problem_options = Options(:vars => [1, 2],
                                  :partition => partition,
                                  :property => property,
                                  :assume_sparse => true,
                                  :lazy_expm => true)
    end

    return (S, merge(problem_options, input_options))
end

mna5 (generic function with 2 methods)

In [9]:
@time p, opts = mna5(:δ => 1e-3, :N => 3, :mode=>"reach", :verbosity => "info"); # warm-up

  0.008737 seconds (196 allocations: 3.330 MiB, 77.07% gc time)


In [10]:
@time sol = solve(p, opts);

[info] Time discretization...
[info] elapsed time: 496.390988934 seconds
[info] Reachable States Computation...
[info] - Decomposing X0
[info] elapsed time: 548.882165666 seconds
[info] - Computing successors
[info] elapsed time: 2.325836262 seconds
[info] - Total
[info] elapsed time: 552.554339634 seconds
[info] Projection...
[info] elapsed time: 0.414895677 seconds
1057.147102 seconds (22.82 M allocations: 277.874 GiB, 2.46% gc time)


In [11]:
using Plots

In [ ]:
plot(sol)